# Attempt to Model Velocity Dependence _with DNNs_
Retry the problem form the previous notebook

In [1]:
%matplotlib inline
import os
import sys
sys.path.append(f"{os.getcwd()}/../")
# os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
from matplotlib import pyplot as plt
from stopping_power_ml.io import get_charge_density_interpolator, load_directory
from stopping_power_ml.keras import build_fn as build_dense
from stopping_power_ml.integrator import TrajectoryIntegrator

from glob import glob
from tqdm import tqdm_notebook as tqdm

from keras.layers import Dense, Activation, Input
from keras.layers import Concatenate, Multiply, Add
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pickle as pkl
import numpy as np
import pandas as pd
import keras
import time

dname = "random_models"
# randint = np.random.randint(0, 101)
# keras.utils.set_random_seed(randint)
keras.backend.set_floatx('float32')
keras.config.set_floatx('float32')
print(keras.config.floatx())

2024-06-21 16:08:26.690393: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 16:08:26.696283: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-21 16:08:26.762059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-21 16:08:31.463623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


float32


## Load in Data
Read in the random trajectories

In [2]:
data = pd.read_pickle(os.path.join('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities', 'random_traj.pkl'))

In [3]:
channel_data = pd.read_pickle(os.path.join('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities', 'channel_traj.pkl')).query('average_range')
random_data = pd.read_pickle(os.path.join('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities', 'random_traj.pkl'))
start_frame = pkl.load(open(os.path.join('..', 'al_starting_frame.pkl'), 'rb'))

In [4]:
with open('featurizer.pkl', 'rb') as fp:
    featurizer = pkl.load(fp)

Get the features

In [5]:
X_cols = featurizer.feature_labels()

In [6]:
X_cols_novel = list(X_cols)
X_cols_novel.remove("velocity_mag")
print(X_cols, '\n',X_cols_novel)

['charge density at t=-4.00', 'charge density at t=-3.00', 'charge density at t=-2.00', 'charge density at t=-1.00', 'charge density at t=-0.50', 'charge density at t=0.00', 'charge density at t=0.50', 'charge density at t=1.00', 'charge density at t=2.00', 'AGNI projected eta=8.00e-01', 'AGNI projected eta=1.23e+00', 'AGNI projected eta=1.88e+00', 'AGNI projected eta=2.89e+00', 'AGNI projected eta=4.43e+00', 'AGNI projected eta=6.80e+00', 'AGNI projected eta=1.04e+01', 'AGNI projected eta=1.60e+01', 'ion-ion repulsion', 'velocity_mag'] 
 ['charge density at t=-4.00', 'charge density at t=-3.00', 'charge density at t=-2.00', 'charge density at t=-1.00', 'charge density at t=-0.50', 'charge density at t=0.00', 'charge density at t=0.50', 'charge density at t=1.00', 'charge density at t=2.00', 'AGNI projected eta=8.00e-01', 'AGNI projected eta=1.23e+00', 'AGNI projected eta=1.88e+00', 'AGNI projected eta=2.89e+00', 'AGNI projected eta=4.43e+00', 'AGNI projected eta=6.80e+00', 'AGNI proje

In [7]:
X_cols_vel = ["velocity_mag"]

In [8]:
y_col = 'force'

## Create Functions for Building Models
Create functions for building a Keras model

In [9]:
input_shape = (len(X_cols_novel),)
v_input_shape = (len(X_cols_vel),)
layers = (input_shape[0],32,24,8,4,3)
v_layers = (v_input_shape[0],3,3)
m_layers = (layers[-1],3)
activation = 'elu'

In [10]:
optimizer_options = {
    'loss': 'mean_absolute_error', 
    'optimizer': Adam(learning_rate=0.0005),
    'metrics': ['mean_absolute_error']
}

In [11]:
callbacks = [EarlyStopping(monitor='mean_absolute_error', patience=100, verbose=0, mode='auto'),
             ReduceLROnPlateau('mean_absolute_error')]

### Standard Dense Model
Just treat all layers equally

In [12]:
model = build_dense(len(X_cols), layers, activation=activation, use_linear_block=False,
                   optimizer_options=optimizer_options)
model.summary()
print(model.get_weights()[0].dtype)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 18)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,015 (7.87 KB)

 Trainable params: 2,015 (7.87 KB)

 Non-trainable params: 0 (0.00 B)

float32


In [13]:

double = keras.models.load_model('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities/random_models/double_model48.h5')
double.summary()
print(double.get_weights()[0].dtype)
# Convert and copy weights and biases to the new double precision model
for layer_double, layer_single in zip(double.layers, model.layers):
    weights_double = layer_double.get_weights()
    weights_single = [w.astype('float32') for w in weights_double]
    layer_single.set_weights(weights_double)

print(model.get_weights()[0].dtype)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 18)             │           360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 24)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            15 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,017 (15.75 KB)

 Trainable params: 2,015 (15.74 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

float64
float32


In [14]:
# Function to compare weights and biases
def compare_weights_and_biases(model1, model2, tol=1e-6):
    for layer1, layer2 in zip(model1.layers, model2.layers):
        weights1 = layer1.get_weights()
        weights2 = layer2.get_weights()
        for w1, w2 in zip(weights1, weights2):
            if not np.allclose(w1, w2, atol=tol):
                print(f"Difference found in layer {layer1.name}")
                return False
    return True
print(compare_weights_and_biases(double, model))

True


In [15]:
model.save(f'{dname}/single_model.h5')

## Make the Trajectory Integration Tool
This will black box the annoying parts of making the trajectories

In [16]:
traj_int = TrajectoryIntegrator(start_frame, model, featurizer)

## Evaluate $\left<100\right>$ at Many Velocities
See how smoothly it varies and whether the difference between channel/not channel increases at higher velocities

In [17]:
channel_stopping = []
for gid, group in channel_data.groupby('directory'):
    channel_stopping.append({
        'velocity': group['velocity_mag'].mean(),
        'stopping': group['force'].mean()
    })
channel_stopping = pd.DataFrame(channel_stopping)

In [18]:
%%time
# velocities = np.linspace(0.1, 5, 22, endpoint = True)
velocities = np.append(np.linspace(0.1, 0.6, 5, endpoint = True), np.linspace(0.8, 5, 15, endpoint = True))
stopping_power = [
    traj_int.compute_stopping_power([0, 0.75, 0.75], [1, 0, 0], v)[0]
    for v in velocities
]

CPU times: user 1min, sys: 461 ms, total: 1min 1s
Wall time: 1min 1s


## Run the Random Trajectory at Many Velocities
We are going to compute the average stopping power by evaluating the stopping force at every position along the trajectory with different speeds

In [19]:
random_stopping = []
for gid, group in random_data.groupby('directory'):
    random_stopping.append({
        'velocity': group['velocity_mag'].mean(),
        'stopping': group['force'].mean()
    })
random_stopping = pd.DataFrame(random_stopping)

In [20]:
def compute_random_stopping_power(speed: float) -> float:
    """Compute the stopping power for the random trajectory
    
    Args:
        speed: Magnitude of the velocity
    Returns:
        Stopping power
    """
    start = time.time()
    # Get the positions along the trajectory
    random_positions = random_data[random_data['file'].str.contains('Dv1.0')].copy()
    
    # Update the velocity
    random_positions['velocity'] *= speed
    
    # Run the model
    featurizer.set_chunksize(None)
    x = featurizer.featurize_many(random_positions[['position', 'velocity']], pbar=False)
    forces = model.predict(np.array(x))
    print("executive time", time.time() - start, 's')
    return forces.mean()

In [21]:
%%time
stopping_power_random = [
    compute_random_stopping_power(v)
    for v in velocities
]

  1/294 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step

 57/294 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step

114/294 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step

172/294 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step

230/294 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step

...

122/294 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step

181/294 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step

240/294 ━━━━━━━━━━━━━━━━━━━━ 0s 841us/step

294/294 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step


executive time 97.64313268661499 s
CPU times: user 1min 29s, sys: 2min 6s, total: 3min 35s
Wall time: 33min 22s


In [22]:
np.savetxt(f'{dname}/double_sp.dat', np.stack([velocities, stopping_power, stopping_power_random]).T, header = "vel channel random")

## retrain the data

In [23]:
train_data = pd.concat([
    pd.read_pickle(os.path.join('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities', 'random_traj.pkl')),
    pd.read_pickle(os.path.join('/scratch/users/pdanie20/PD-stopping-power-ml/multiple-velocities', 'channel_traj.pkl'))
], sort=False)
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
print(model.get_weights()[0].dtype)
callbacks = [keras.callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100, verbose=0, mode='auto'),
             keras.callbacks.ReduceLROnPlateau('mean_absolute_error', min_delta=1e-6)]

float64


In [24]:
%%time
history = model.fit(train_data[featurizer.feature_labels()], train_data['force'], callbacks=callbacks, verbose=0, epochs=1000)
print(model.get_weights()[0].dtype)

# Check if early stopping criteria were met
if callbacks[0].stopped_epoch > 0:
    print(f"Early stopping triggered after epoch {callbacks[0].stopped_epoch}")
else:
    print("Early stopping was not triggered")
    exit()

float64
Early stopping triggered after epoch 478
CPU times: user 16min 51s, sys: 27.6 s, total: 17min 18s
Wall time: 20min 21s


In [25]:
model.save(f'{dname}/double_model_retrained.h5')

In [26]:
%%time
# velocities = np.linspace(0.1, 5, 22, endpoint = True)
velocities = np.append(np.linspace(0.1, 0.6, 5, endpoint = True), np.linspace(0.8, 5, 15, endpoint = True))
stopping_power = [
    traj_int.compute_stopping_power([0, 0.75, 0.75], [1, 0, 0], v)[0]
    for v in velocities
]
%%time
stopping_power_random = [
    compute_random_stopping_power(v)
    for v in velocities
]

UsageError: Line magic function `%%time` not found.


In [27]:
np.savetxt(f'{dname}/double_sp_retrained.dat', np.stack([velocities, stopping_power, stopping_power_random]).T, header = "vel channel random")